## Imports And Set Up

In [8]:
import pandas as pd
import numpy as np
import os
from scipy.spatial import cKDTree
from datetime import datetime

os.makedirs("../data/summary_stats", exist_ok=True)

## Summary Table

### Check Missing Values

In [ ]:
# === Load and normalize ===
df = pd.read_csv("../data/raw/crash_data_2014_to_2024.csv")
df.columns = df.columns.str.strip().str.lower()


# === 1️⃣ Remove NaN entries in key vars ===
key_cols = [
    "county_id",
    "crash_severity_id",
    "light_condition_id",
    "weather_condition_id",
    "roadway_surf_condition_id"
]
df_clean = df.dropna(subset=key_cols)

# === 2️⃣ Remove invalid category codes ===
invalid_codes = [8, 89, 96, 97, 98, 99, 88, "U", "Unknown", "Not Provided", "Invalid"]
for col in key_cols[1:]:  # skip county_id
    df_clean = df_clean[~df_clean[col].isin(invalid_codes)]

print(f"Original rows: {len(df)}")
print(f"Rows after full filtering: {len(df_clean)}")


Original rows: 272524
Rows after full filtering: 267888


### Create Summary CSV

In [25]:
# === County ID → Name mapping ===
county_map = {
    11: "Davis",
    35: "Salt Lake",
    49: "Utah",
    53: "Washington",
    57: "Weber"
}

df["county_name"] = df["county_id"].map(county_map)
df = df[df["county_name"].notna()]  # keep only the 5 target counties

# === Variable label mappings ===
severity_map = {
    1: "No injury / PDO",
    2: "Possible injury",
    3: "Suspected Minor Injury",
    4: "Suspected Serious Injury",
    5: "Fatal"
}

light_map = {
    1: "Daylight",
    2: "Dark - Lighted",
    3: "Dark - Not Lighted",
    4: "Dark - Unknown Lighting",
    5: "Dawn",
    6: "Dusk"
}

weather_map = {
    1: "Clear",
    2: "Cloudy",
    3: "Rain",
    4: "Snowing",
    5: "Blowing Snow",
    6: "Sleet, Hail",
    7: "Fog, Smog",
    8: "Severe Crosswinds",
    9: "Blowing Sand / Dirt"
}

road_map = {
    1: "Dry",
    2: "Wet",
    3: "Snow",
    4: "Slush",
    5: "Ice / Frost",
    6: "Water",
    7: "Mud",
    8: "Sand / Dirt / Gravel",
    9: "Oil",
    10: "Dirt",
    11: "Gravel",
    12: "Sand",
    97: "Other"
}

# === Helper function to make county-level proportion table ===
def make_summary(df, var, mapping, var_label):
    tmp = df[[var, "county_name"]].copy()
    tmp[var] = tmp[var].map(mapping)

    # Count occurrences by county and category
    counts = tmp.groupby(["county_name", var]).size().unstack(fill_value=0)

    # Convert to proportions (row sums = 1 per county)
    props = counts.div(counts.sum(axis=1), axis=0).T.reset_index()

    # The first column now contains category labels — name it explicitly
    props.rename(columns={props.columns[0]: "Category"}, inplace=True)

    # Add the numeric code dynamically using the mapping dictionary
    label_to_code = {v: k for k, v in mapping.items()}
    props.insert(0, "Code", props["Category"].map(label_to_code))

    # Fill any missing labels as 'Unknown'
    props["Category"] = props["Category"].fillna("Unknown")

    return props


# === Generate summaries ===
severity_summary = make_summary(df, "crash_severity_id", severity_map, "Crash Severity")
light_summary = make_summary(df, "light_condition_id", light_map, "Light Condition")
weather_summary = make_summary(df, "weather_condition_id", weather_map, "Weather Condition")
road_summary = make_summary(df, "roadway_surf_condition_id", road_map, "Roadway Surface Condition")

# === Combine all summaries into one DataFrame ===
severity_summary["Variable"] = "Crash Severity"
light_summary["Variable"] = "Light Condition"
weather_summary["Variable"] = "Weather Condition"
road_summary["Variable"] = "Road Surface"

combined = pd.concat(
    [severity_summary, light_summary, weather_summary, road_summary],
    ignore_index=True
)

# Move "Variable" to the front
cols = ["Variable", "Code", "Category"] + [c for c in combined.columns if c not in ["Variable", "Code", "Category"]]
combined = combined[cols]

# === Save one combined CSV ===
combined.to_csv("../data/summary_stats/summary_all_variables.csv", index=False)


### Add Standard Errors

In [27]:
# === Load your combined summary file ===
df = pd.read_csv("../data/summary_stats/summary_all_variables.csv")

# === Total number of observations (Salt Lake crashes) ===
n = 267888

# === Compute binomial standard error for each proportion ===
# Formula: SE = sqrt(p * (1 - p) / n)
df["SE"] = np.sqrt(df["Salt Lake"] * (1 - df["Salt Lake"]) / n)

# === Optional: round for readability ===
df["Salt Lake"] = df["Salt Lake"].round(6)
df["SE"] = df["SE"].round(6)

# === Save new version ===
df.to_csv("../data/summary_stats/summary_all_variables_with_SE.csv", index=False)


## Other Numbers

### Crashes Per Day

In [ ]:
# === Load data ===
df = pd.read_csv("../data/raw/crash_data_2014_to_2024.csv")

# === Convert datetime column to actual datetime objects ===
df["crash_datetime"] = pd.to_datetime(df["crash_datetime"], errors="coerce")

# === Extract date (YYYY-MM-DD) ===
df["crash_date"] = df["crash_datetime"].dt.date

# === Count distinct crash IDs per day ===
daily_crashes = (
    df.groupby("crash_date")["crash_id"]
    .nunique()               # count distinct crash IDs
    .reset_index(name="crash_count")
    .sort_values("crash_date")
)

# === Show summary ==
print(f"\nComputed daily crash counts for {len(daily_crashes)} days.")
print("Average crashes per day:", daily_crashes["crash_count"].mean())
print("Standard deviation:", daily_crashes["crash_count"].std())




Computed daily crash counts for 4018 days.
Average crashes per day: 67.82578397212544
Standard deviation: 25.261017525385753


## Merge Pipeline

### Create Vehicle Size Column

In [ ]:
# Load your data
vehicle = pd.read_csv("../data/raw/vehicle_data_2020_2025.csv", low_memory=False)

# --- Mapping from mv_body_text to simplified category ---
body_map = {
    # Cars
    "Passenger Car (2 door) (Renamed Passenger Car in 2019)": "sedan",
    "Passenger Car (4 door) (removed 2019)": "sedan",
    "Station Wagon (removed 2019)": "sedan",
    "Limousine (new 2019)": "sedan",

    # SUVs / Crossovers
    "Sport Utility Vehicle": "suv",

    # Trucks
    "Pickup": "truck",
    "Single Unit Truck (2 axles 6 tires) (Renamed Single Unit Truck in 2019)": "truck",
    "Single Unit Truck (3 or more axles) (removed 2019)": "truck",
    "Truck Tractor": "truck",
    "Heavy Truck Other": "truck",
    "Construction Equip.(backhoe,bulldozer,etc) (new 2019)": "truck",

    # Vans
    "Cargo Van (new 2019)": "van",
    "Passenger Van (15 seats) (new 2019)": "van",
    "Passenger Van (9-12 seats) (new 2019)": "van",
    "Passenger Van (<9 seats) (new 2019)": "van",
    "Van or Mini Van (removed 2019)": "van",
    "RV/Motor Home": "van",

    # Buses
    "School Bus": "bus",
    "Transit Bus (new 2019)": "bus",
    "Other Bus Type* (new 2019)": "bus",
    "Bus/Motorcoach (not school) (removed 2019)": "bus",
    "Motorcoach (new 2019)": "bus",

    # Motorcycles and similar
    "Motorcycle (2 wheels) (new 2019)": "motorcycle",
    "Motorcycle (3 wheels) (new 2019)": "motorcycle",
    "Motorcycle (removed 2019)": "motorcycle",
    "Motorized Scooter/Moped etc.": "motorcycle",

    # ATVs / Off-road / Snow
    "ATV Street Legal": "offroad",
    "ATV/OV Off road (new 2019)": "offroad",
    "Off Road Vehicle (snowmobile, ATV, etc.) (removed 2019)": "offroad",
    "Snowmobile (new 2019)": "offroad",
    "Golft Cart (new 2019)": "offroad",
    "Farm Equipment (combine, etc.)": "offroad",

    # Unknown / other
    "Other*": "other",
    "Unknown": "other"
}

# --- Apply the mapping ---
vehicle["vehicle_size"] = vehicle["mv_body_text"].map(body_map).fillna("other")

# --- One-hot encode vehicle types and sum by crash_id ---
type_counts = pd.get_dummies(vehicle["vehicle_size"])
type_counts["crash_id"] = vehicle["crash_id"]

vehicle_type_summary = (
    type_counts.groupby("crash_id")
    .sum()
    .reset_index()
)

vehicle_type_summary

### Merge Vehicle Data Into Crash Data

In [13]:
crash = pd.read_csv("../data/raw/crash_data_2020_2025.csv", low_memory=False)

# --- Convert datetime columns ---
crash["crash_datetime"] = pd.to_datetime(crash["crash_datetime"], errors="coerce")
vehicle["crash_datetime"] = pd.to_datetime(vehicle["crash_datetime"], errors="coerce")

# 1. Aggregate VEHICLE data to crash level

# Example aggregate (keep one row per crash)
agg_funcs = {
    "vehicle_num": "count",
    "vehicle_year": "mean",
    "estimated_travel_speed": "mean",
    "posted_speed": "mean"
}

# Treat impossible vehicle_years as missing
vehicle.loc[vehicle["vehicle_year"] <= 1900, "vehicle_year"] = np.nan

vehicle_summary = (
    vehicle.groupby("crash_id")
    .agg(agg_funcs)
    .reset_index()
    .rename(columns={
        "vehicle_num": "num_vehicles_reported",
        "vehicle_year": "avg_vehicle_year",
        "estimated_travel_speed": "avg_estimated_speed",
        "posted_speed": "avg_posted_speed"
    })
)

# 2. Merge vehicle summary into crash

# --- merge summaries together ---
vehicle_merged = vehicle_summary.merge(vehicle_type_summary, on="crash_id", how="left")

# --- merge into crash data ---
crash_veh = crash.merge(vehicle_merged, on="crash_id", how="left")

crash_veh

,crash_id,crash_datetime,year,month,day,hour,weekday,date,region_id,county_id,...,avg_estimated_speed,avg_posted_speed,bus,motorcycle,offroad,other,sedan,suv,truck,van
0,820606896,2020-01-25 17:17:00,2020,1,25,17,7,2020-01-25,3,49,...,17.5,25.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,820619625,2020-04-07 17:21:00,2020,4,7,17,3,2020-04-07,2,35,...,0.0,40.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
2,820638169,2020-08-17 17:19:00,2020,8,17,17,2,2020-08-17,1,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,820613836,2020-02-25 20:01:00,2020,2,25,20,3,2020-02-25,2,35,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,820626411,2020-06-01 16:12:00,2020,6,1,16,2,2020-06-01,3,49,...,20.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291477,824905553,2024-05-02 14:34:00,2024,5,2,14,5,2024-05-02,3,49,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
291478,824919437,2024-10-05 11:01:00,2024,10,5,11,7,2024-10-05,1,11,...,35.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
291479,824919584,2024-10-05 19:45:00,2024,10,5,19,7,2024-10-05,3,49,...,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
291480,824919585,2024-10-01 18:10:00,2024,10,1,18,3,2024-10-01,2,35,...,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


### Merge Weather Data Into Crash Data

In [ ]:
# --- Load and prep weather data ---
weather = pd.read_csv("utah_weather_test.csv", low_memory=False)
weather["date_time"] = pd.to_datetime(weather["date_time"], errors="coerce")

# --- Build KDTree for nearest station ---
station_coords = weather[["latitude", "longitude"]].drop_duplicates().to_numpy()
station_tree = cKDTree(station_coords)

# --- For each crash, find nearest weather station ---
crash_coords = crash_veh[["lat", "long"]].to_numpy()
distances, station_idx = station_tree.query(crash_coords, k=1)

nearest_stations = (
    weather[["latitude", "longitude"]]
    .drop_duplicates()
    .iloc[station_idx]
    .reset_index(drop=True)
)

crash_veh["nearest_station_lat"] = nearest_stations["latitude"].values
crash_veh["nearest_station_lon"] = nearest_stations["longitude"].values
crash_veh["station_distance_km"] = distances * 111  # approx. km per degree

# --- Function to get the nearest-in-time weather record for each crash ---
def get_closest_weather(row):
    subset = weather[
        (weather["latitude"] == row["nearest_station_lat"]) &
        (weather["longitude"] == row["nearest_station_lon"])
    ]
    if subset.empty:
        # return one NaN for every column in weather except date_time
        return pd.Series([np.nan] * (weather.shape[1] - 1), index=weather.columns.drop("date_time"))
    # find the row in this station's data with datetime closest to the crash
    idx = (subset["date_time"] - row["crash_datetime"]).abs().idxmin()
    # return every column except the date_time itself
    return subset.loc[idx, weather.columns.drop("date_time")]

# --- Apply matching ---
weather_matched = crash_veh.apply(get_closest_weather, axis=1)

# --- Concatenate back with crash data ---
crash_full = pd.concat([crash_veh.reset_index(drop=True),
                        weather_matched.reset_index(drop=True)], axis=1)

### Cleanup and Save

In [ ]:
# =====================================================
# 4. Clean + rename + drop irrelevant columns
# =====================================================
keep_cols = [
    # Crash info
    "crash_id", "crash_datetime", "year", "month", "day", "hour", "weekday",
    "city", "county_text",
    "crash_severity_text", "light_condition_text", "weather_condition_text", "roadway_surf_condition_text",
    
    # Vehicle aggregates
    "num_vehicles_reported", "avg_vehicle_year", "avg_estimated_speed", "avg_posted_speed",
    "sedan", "suv", "truck", "van", "motorcycle", "bus", "offroad", "other",
    
    # Weather variables
    "air_temp_set_1", "relative_humidity_set_1", "wind_speed_set_1", "wind_gust_set_1",
    "precip_accum_one_hour_set_1", "snow_depth_set_1",
    "visibility_set_1", "dew_point_temperature_set_1d", "wind_direction_set_1",
    "pressure_set_1", "solar_radiation_set_1"
]

cleaned = crash_full[keep_cols].copy()

# --- Rename for clarity ---
cleaned.rename(columns={
    "crash_severity_text": "severity",
    "light_condition_text": "light_condition",
    "weather_condition_text": "weather_condition",
    "roadway_surf_condition_text": "road_surface",
    "air_temp_set_1": "temp_C",
    "relative_humidity_set_1": "humidity_pct",
    "wind_speed_set_1": "wind_speed_mps",
    "wind_gust_set_1": "wind_gust_mps",
    "precip_accum_one_hour_set_1": "precip_mm",
    "snow_depth_set_1": "snow_depth_mm",
    "visibility_set_1": "visibility_m",
    "dew_point_temperature_set_1d": "dew_point_C",
    "wind_direction_set_1": "wind_dir_deg",
    "pressure_set_1": "pressure_pa",
    "solar_radiation_set_1": "solar_rad_wm2"
}, inplace=True)

# 5. Save cleaned dataset
cleaned.to_csv("merged_crash_vehicle_weather_clean.csv", index=False)
print("✅ Clean dataset saved: merged_crash_vehicle_weather_clean.csv")

# # Optional quick shape + sample check
# print("Final shape:", cleaned.shape)
# print(cleaned.head(3))
